**Criando conexão no sistema.**

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session
from pyspark.sql import functions as F

# This will create a default Spark session
spark = DataprocSparkSession.builder.getOrCreate()

Using existing Dataproc Session (configuration changes may not be applied): https://console.cloud.google.com/dataproc/interactive/northamerica-northeast1/sc-20251015-174547-7wvwwm?project=clinic-de


**Lendo tabela da bronze**

In [ ]:
df = spark.read.format("bigquery").option("table", "clinic-de.1_bronze.patients").load()

df.printSchema()


root
 |-- metadata: struct (nullable = true)
 |    |-- blood_type: string (nullable = true)
 |-- insurance: struct (nullable = true)
 |    |-- card_number: long (nullable = true)
 |    |-- plan_name: string (nullable = true)
 |    |-- has_plan: boolean (nullable = true)
 |-- contacts: struct (nullable = true)
 |    |-- phones: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- email: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- document: struct (nullable = true)
 |    |-- cpf: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- zip: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- geo: struct (nullable = true)
 |    |    |-- lng: double (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |-- street: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- birth_date:

**Selecionando colunas do dataframe**

In [ ]:
df_tratado_v1 = (df.select(
    "patient_id",
    "metadata.blood_type",
    "insurance.*",
    "contacts.*",
    "address.*",
    "sex",
    "birth_date",
    "name"

    )
)

**Dropando coluna**

In [ ]:
df_tratado_v2 = df_tratado_v1.select("*", "geo.*").drop("geo")

**Substituindo valores nulos da coluna card_number**

---



In [ ]:
df_tratado_v2 = df_tratado_v2.fillna(0, subset=["card_number"])

**Substituindo valores nulos da coluna "plan_name" e "card_number".**

In [ ]:
df_tratado_v2 = df_tratado_v2.withColumn(
                                        "plan_name",
                                         F.when((F.col("plan_name").isNull()) | (F.col("card_number") == 0), "DESCONHECIDO")
                                         .otherwise(F.col("plan_name"))
                                        )

**Substituindo nulos por 0 na coluna "card_number".**

In [ ]:
df_tratado_v3 = df_tratado_v2.withColumn(
                         "has_plan",
                         F.when(F.col("card_number") == 0, False )
                         .otherwise(True)
)

In [ ]:
df_tratado_v4 = df_tratado_v3.withColumn("phones", F.col("phones").getItem(0))
df_tratado_v4.show()

+----------+----------+-----------+------------+--------+-------------------+--------------------+---------+-------+-----+------------------+--------------------+---+----------+--------------------+-----------+-----------+
|patient_id|blood_type|card_number|   plan_name|has_plan|             phones|               email|      zip|country|state|              city|              street|sex|birth_date|                name|        lng|        lat|
+----------+----------+-----------+------------+--------+-------------------+--------------------+---------+-------+-----+------------------+--------------------+---+----------+--------------------+-----------+-----------+
|PAT-000175|        A+| 6022457143|  SULAMERICA|    true|+55 (041) 0243 1588|cirinonicole@yaho...| 16152959|     BR|   AC|da Cruz das Flores|Viaduto de Vascon...|  F|1946-11-08|       Yasmin Garcia| -16.505322| 21.2266455|
|PAT-001409|        A+| 9609495991|    BRADESCO|    true|   +55 21 1532-1605|abreuayla@hotmail...| 12730109|

**Verificando valores nulos de todas colunas**

In [ ]:
df_tratado_v4.select([F.sum(F.col(c).isNull().cast("integer")).alias(c) for c in df_tratado_v4.columns]).show()


+----------+----------+-----------+---------+--------+------+-----+---+-------+-----+----+------+---+----------+----+---+---+
|patient_id|blood_type|card_number|plan_name|has_plan|phones|email|zip|country|state|city|street|sex|birth_date|name|lng|lat|
+----------+----------+-----------+---------+--------+------+-----+---+-------+-----+----+------+---+----------+----+---+---+
|         0|         0|          0|        0|       0|     0|    0|  0|      0|    0|   0|     0|  0|         0|   0|  0|  0|
+----------+----------+-----------+---------+--------+------+-----+---+-------+-----+----+------+---+----------+----+---+---+



**Validando max e minimo da coluna**

In [ ]:
min_max_select = df_tratado_v4.select(F.min("birth_date").alias("min_valor"), F.max("birth_date").alias("max_valor"))
min_max_select.show()

+----------+----------+
| min_valor| max_valor|
+----------+----------+
|1929-12-07|2025-10-09|
+----------+----------+



**removendo todas as abreviações/títulos terminados em ponto no começo do nome com O regex.**

In [ ]:
df_tratado_v5 = (
    df_tratado_v4
    .withColumn(
        "name",
        F.trim(F.regexp_replace(F.col("name"), r"^(?:\s*\S+\.\s+)+", ""))
    )
)


**Salvando dataframe na silver.**

In [ ]:
(df_tratado_v5.write
        .format("bigquery")
        .option("table", "clinic-de.2_silver.patients")
        .option("writeMethod", "direct")
        .mode("overwrite")
        .save()
    )